In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import time
import re

<IPython.core.display.Javascript object>

In [3]:
# load in scraped tweets from https://github.com/tarrantcarter/Final_Capstone/blob/main/Modern_Motivation_Web_Scrape.ipynb
tweets = pd.read_csv(
    r"/Users/tarrantcarter/Dropbox/Data Science/Final_Capstone/text_query_tweets.csv"
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tarrantcarter/Dropbox/Data Science/Final_Capstone/text_query_tweets.csv'

<IPython.core.display.Javascript object>

In [ ]:
tweets.head()

In [ ]:
tweets.info()

In [ ]:
tweets.applymap(type)

In [ ]:
# extend max col width so all the text can be seen
pd.set_option("display.max_colwidth", None)

tweets["content"][:20]

In [ ]:
tweets["content"][:-10]

After further inspection of the content column, tweets that start with @ are specific tweets to other user. These tweets will be removed becuase we want motinational tweets that are geared towards a general audience. We also have to remove tweets that have hyperlinks since those tweets are more calls to action and not general motivational advise. We will also have to deal with emojis.  

In [ ]:
# convert any non-text to text in the content column
tweets["content"] = tweets["content"].map(str)

In [ ]:
# remove all tweets that contain @
tweets2 = tweets[~tweets['content'].str.contains("@")]


In [ ]:
tweets2.info()

That removed about 180,000 rows. 

In [ ]:
tweets2["content"][:20]

In [ ]:
# drop rows containing http links
tweets3 = tweets2[~(tweets2["content"].str.contains("http"))]

In [ ]:
tweets3["content"][:20]

In [ ]:
len(tweets3)

We have removed about a quarter of our initial database. We still need to remove line breaks, non-english text, emojis, and emoticons.

In [ ]:
tweets3.info()

In [ ]:
# remove line breaks and change non-latin apostrophe to latin counterpart
tweets3["content"] = tweets3["content"].str.replace("\n", " ").str.replace("’", "'")

In [ ]:
tweets3["content"][:20]

Next we will remove text that contains more than one character outside of the latin alphabet. This will include emojis and emoticons by taking advantage of the ord() function. This step won't completely remove all non-english tweets, but it will deal with emojis and emoticons. Languages like French, German, and Spanish use a latin alphabet. We will remove the rest of the non-english text later. 

In [ ]:
# function that returns True if no more than one character is non-latin
def is_latin_alphabet(string):
    non_ascii = 0

    for character in string:
        if ord(character) > 127:
            non_ascii += 1

    if non_ascii > 1:
        return False
    else:
        return True

In [ ]:
# make a dictionary with emoji less text or append with none
latin_emojiless_text = []

for text in tweets3["content"]:
    if is_latin_alphabet(text) == True:
        latin_emojiless_text.append(text)
    else:
        latin_emojiless_text.append(None)


len(latin_emojiless_text)

In [ ]:
# save emojiless text to new dataframe
tweets4 = tweets3.copy()
tweets4["content"] = latin_emojiless_text

In [ ]:
tweets4.head()

In [ ]:
tweets4["content"][:-20]

We do see we will have to remove hashtage as well from our tweets. 

In [ ]:
def remove_hashtags(text):
    text = re.sub("#\S+", "", str(text))
    return text

In [ ]:
tweets5 = tweets4.copy()

no_hashtags = [remove_hashtags(text) for text in tweets5["content"]]

In [ ]:
tweets5["content"] = no_hashtags

tweets5["content"][:-20]

In [ ]:
tweets5.shape

In [ ]:
tweets5["lang"].value_counts()

In [ ]:
# remove non-english tweets
tweets6 = tweets5[tweets["lang"] == "en"]

tweets6["lang"].value_counts()

Next, we will remove unnecessary columns for our chatbot. The only columns we need are the content column, a column that identifies the author of the tweet, and the date. 

In [ ]:
tweets6.head()

In [ ]:
# condense dataframe to only the 3 necessary columns
tweets7 = tweets6[["date", "content", "user"]]

tweets7.head()

In [ ]:
tweets7["user_name"]

In [ ]:
# extract username and display names to seperate columns
user_name = "username"
display_name = ["displayname"]

tweets7["user_name"] = [
    value[user_name]
    for (key, value) in tweets7["user"].items()
    if user_name in value
]

In [ ]:
tweets7.applymap(type)

In [ ]:
from ast import literal_eval


def try_literal_eval(s):
    try:
        return literal_eval(s)
    except ValueError:
        return s

In [ ]:
tweets7["user"].apply(try_literal_eval)

In [ ]:
tweets7.applymap(type)

In [ ]:
tweets7.head()

In [ ]:
# extract username and display names to seperate columns
user_name = ["username"]
display_name = ["displayname"]

tweets7["user_name"] = [
    value[user_name] for (key, value) in tweets7["user"].items() if user_name in value
]

In [ ]:
for key, value in tweets7["user"][:1].items():
    value.apply(eval)
#     for k, v in value:
#         if k == "username":
#             print(v)

In [ ]:
print(tweets7["user"].values[0])

In [ ]:
items = list(tweets7["user"].items())
print(items[0][1])

In [ ]:
# remove cuss words

In [ ]:
# tweet bot will have a drop down for topic and seperate one for twitter user account


In [ ]:
json = pd.read_json(
    r"/Users/tarrantcarter/Dropbox/Data Science/Final_Capstone/motivational_tweets.json",
    lines=True,
)

In [ ]:
json.head()

In [ ]:
json.applymap(type)

In [ ]:
json.info()

In [ ]:
# extract username and display names to seperate columns
user_name = ["username"]
display_name = ["displayname"]

json["user_name"] = [
    value[user_name] for (key, value) in tweets7["user"].items() if user_name in value
]